In [1]:
import pandas as pd
import numpy as np
import time



df = pd.read_csv('household_power_consumption.txt', delimiter=';', low_memory=False)
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [2]:
df= df.drop(df[df.isin(['?']).any(axis=1)].index)
columns_to_convert = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2']
df[columns_to_convert] = df[columns_to_convert].astype(float)
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
#df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S')
df.dtypes
df.head()



,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,2006-12-16,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,2006-12-16,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,2006-12-16,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,2006-12-16,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


1. Обрати всі домогосподарства, у яких загальна активна споживанапотужність перевищує 5 кВт.

In [3]:
start_time = time.time()
active_power = df[df["Global_active_power"] > 5]
execution_time = time.time() - start_time
print("Час виконання алгоритму: %s секунд" % execution_time)

active_power.head()

execution_time_pandas = []
execution_time_pandas.append(execution_time)





Час виконання алгоритму: 0.12074685096740723 секунд


2. Обрати всі домогосподарства, у яких вольтаж перевищую 235 В.

In [4]:
start_time = time.time()
voltage = df[df["Voltage"] > 235]
execution_time = time.time() - start_time
print("Час виконання алгоритму: %s секунд" % execution_time)
voltage.head()

execution_time_pandas.append(execution_time)


Час виконання алгоритму: 0.13515734672546387 секунд


3. Обрати всі домогосподарства, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер.

In [5]:
start_time = time.time()
intensity= df[(df["Global_intensity"] > 19)&(df["Global_intensity"] < 20)]
sub_met2 = intensity[(intensity["Sub_metering_2"]) > (intensity["Sub_metering_3"]) ] 
execution_time = time.time() - start_time
print("Час виконання алгоритму: %s секунд" % execution_time)
sub_met2.head()

execution_time_pandas.append(execution_time)



Час виконання алгоритму: 0.014146566390991211 секунд


4.Обрати випадковим чином 500000 домогосподарств (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії

In [6]:
start_time = time.time()
random_indexes = np.random.choice(df.index, size=500000, replace=True)
random_households = df.loc[random_indexes]
mean_sub_metering_1 = random_households['Sub_metering_1'].mean()
mean_sub_metering_2 = random_households['Sub_metering_2'].mean()
mean_sub_metering_3 = random_households['Sub_metering_3'].mean()
execution_time = time.time() - start_time
print("Час виконання алгоритму: %s секунд" % execution_time)
print("Середнє споживання електричної енергії для Sub_metering_1:", mean_sub_metering_1)
print("Середнє споживання електричної енергії для Sub_metering_2:", mean_sub_metering_2)
print("Середнє споживання електричної енергії для Sub_metering_3:", mean_sub_metering_3)


execution_time_pandas.append(execution_time)


Час виконання алгоритму: 0.7584390640258789 секунд
Середнє споживання електричної енергії для Sub_metering_1: 1.116072
Середнє споживання електричної енергії для Sub_metering_2: 1.292458
Середнє споживання електричної енергії для Sub_metering_3: 6.457508


5. Обрати ті домогосподарства, які після 18-00 споживають понад 6
кВт за хвилину в середньому, серед відібраних визначити ті, у яких
основне споживання електроенергії у вказаний проміжок часу
припадає на пральну машину, сушарку, холодильник та освітлення
(група 2 є найбільшою), а потім обрати кожен третій результат із
першої половини та кожен четвертий результат із другої половини.

In [7]:
start_time = time.time()

active_power = df[(df['Time'] >= '18:00:00') & (df['Global_active_power'] > 6)]
act_power_gr2  = active_power[(active_power["Sub_metering_2"] > active_power["Sub_metering_1"])
                              &(active_power["Sub_metering_2"]> active_power["Sub_metering_3"])]
first_half = act_power_gr2.iloc[:len(act_power_gr2)//2]
second_half = act_power_gr2.iloc[len(act_power_gr2)//2:] 
result = pd.concat([first_half.iloc[::3], second_half.iloc[::4]])
execution_time = time.time() - start_time
print("Час виконання алгоритму: %s секунд" % execution_time)

result.head()

execution_time_pandas.append(execution_time)


Час виконання алгоритму: 0.16016793251037598 секунд


<font size="5">**Виконання завдань з numpy array**</font>

In [8]:
import numpy as np

datanp = np.genfromtxt('household_power_consumption.txt', delimiter=";", dtype=str)



In [10]:

datanp = datanp[1:]  

mask = np.any(datanp == "?", axis=1)

datanp = datanp[~mask]

<font size="4">**1. Обрати всі домогосподарства, у яких загальна активна споживана
потужність перевищує 5 кВт.**</font>

In [11]:
import time

start_time = time.time()
third_column = datanp[:, 2].astype(float)
households_over_5kvt = datanp[third_column > 5]
execution_time = time.time() - start_time

print(households_over_5kvt.shape)
print(households_over_5kvt[:3])

execution_time_numpy = []
execution_time_numpy.append(execution_time)

(17547, 9)
[['16/12/2006' '17:25:00' '5.360' '0.436' '233.630' '23.000' '0.000'
  '1.000' '16.000']
 ['16/12/2006' '17:26:00' '5.374' '0.498' '233.290' '23.000' '0.000'
  '2.000' '17.000']
 ['16/12/2006' '17:27:00' '5.388' '0.502' '233.740' '23.000' '0.000'
  '1.000' '17.000']]


<font size="4">**2. Обрати всі домогосподарства, у яких вольтаж перевищую 235 В.**</font>

In [12]:
start_time = time.time()
voltage_over_235 = datanp[datanp[:,4].astype(float) > 235]
execution_time = time.time() - start_time
print("Час виконання алгоритму: %s секунд" % execution_time)
print(voltage_over_235.shape)
print(voltage_over_235[:5])
execution_time_numpy.append(execution_time)


Час виконання алгоритму: 1.986511468887329 секунд
(1952491, 9)
[['16/12/2006' '17:28:00' '3.666' '0.528' '235.680' '15.800' '0.000'
  '1.000' '17.000']
 ['16/12/2006' '17:29:00' '3.520' '0.522' '235.020' '15.000' '0.000'
  '2.000' '17.000']
 ['16/12/2006' '17:30:00' '3.702' '0.520' '235.090' '15.800' '0.000'
  '1.000' '17.000']
 ['16/12/2006' '17:31:00' '3.700' '0.520' '235.220' '15.800' '0.000'
  '1.000' '17.000']
 ['16/12/2006' '17:38:00' '4.054' '0.422' '235.240' '17.600' '0.000'
  '1.000' '17.000']]


<font size="4">**3. Обрати всі домогосподарства, у яких сила струму лежить в межах
19-20 А, для них виявити ті, у яких пральна машина та холодильних
споживають більше, ніж бойлер та кондиціонер.**</font>

In [13]:
start_time = time.time()
intensity = datanp[(datanp[:,5].astype(float) > 19)&(datanp[:,5].astype(float) < 20)]
execution_time = time.time() - start_time
print("Час виконання алгоритму: %s секунд" % execution_time)
print(intensity.shape)
print(intensity[:5])
execution_time_numpy.append(execution_time)

Час виконання алгоритму: 2.6711955070495605 секунд
(4707, 9)
[['16/12/2006' '17:34:00' '4.448' '0.498' '232.860' '19.600' '0.000'
  '1.000' '17.000']
 ['16/12/2006' '17:48:00' '4.474' '0.000' '234.960' '19.400' '0.000'
  '0.000' '17.000']
 ['16/12/2006' '17:57:00' '4.512' '0.000' '233.620' '19.200' '0.000'
  '0.000' '17.000']
 ['16/12/2006' '18:16:00' '4.524' '0.076' '234.200' '19.600' '0.000'
  '9.000' '17.000']
 ['16/12/2006' '18:18:00' '4.472' '0.000' '233.290' '19.200' '0.000'
  '1.000' '16.000']]


<font size="4">**4. Обрати випадковим чином 500000 домогосподарств (без повторів
елементів вибірки), для них обчислити середні величини усіх 3-х
груп споживання електричної енергії**</font>

In [14]:
start_time = time.time()

random_indexes = np.random.choice(len(datanp), size=500000, replace=True)
random_households = datanp[random_indexes]

mean_sub1 = np.mean(random_households[:,6].astype(float))
mean_sub2 = np.mean(random_households[:,7].astype(float))
mean_sub3 = np.mean(random_households[:,8].astype(float))

execution_time = time.time() - start_time

print("Час виконання алгоритму: %s секунд" % execution_time)
print("Середнє споживання електричної енергії для Sub_metering_1:", mean_sub1)
print("Середнє споживання електричної енергії для Sub_metering_2:", mean_sub2)
print("Середнє споживання електричної енергії для Sub_metering_3:", mean_sub3)
execution_time_numpy.append(execution_time)


Час виконання алгоритму: 2.0261011123657227 секунд
Середнє споживання електричної енергії для Sub_metering_1: 1.126976
Середнє споживання електричної енергії для Sub_metering_2: 1.30593
Середнє споживання електричної енергії для Sub_metering_3: 6.467156


<font size="4">**5. Обрати ті домогосподарства, які після 18-00 споживають понад 6
кВт за хвилину в середньому, серед відібраних визначити ті, у яких
основне споживання електроенергії у вказаний проміжок часу
припадає на пральну машину, сушарку, холодильник та освітлення
(група 2 є найбільшою), а потім обрати кожен третій результат із
першої половини та кожен четвертий результат із другої половини.**</font>

In [15]:
filtered_data = datanp[(datanp[:, 1] > '18:00:00') & (datanp[:, 2].astype(float) > 6)]

selected_data = filtered_data[(filtered_data[:, 7].astype(float) > filtered_data[:, 6].astype(float))
& (filtered_data[:, 7].astype(float) > filtered_data[:, 8].astype(float))]

first_half = selected_data[:len(selected_data) // 2]
second_half = selected_data[len(selected_data) // 2:]

every_third = first_half[::3]
every_fourth = second_half[::4]

result = np.concatenate([every_third, every_fourth])

print(result[0:3])

execution_time_numpy.append(execution_time)

[['16/12/2006' '18:05:00' '6.052' '0.192' '232.930' '26.200' '0.000'
  '37.000' '17.000']
 ['16/12/2006' '18:08:00' '6.308' '0.116' '232.250' '27.000' '0.000'
  '36.000' '17.000']
 ['28/12/2006' '20:58:00' '6.386' '0.374' '236.630' '27.000' '1.000'
  '36.000' '17.000']]


In [16]:
table = pd.DataFrame(columns=["numpy", "pandas"])

table["numpy"] = execution_time_numpy
table["pandas"] = execution_time_pandas

table

,numpy,pandas
0,1.356234,0.120747
1,1.986511,0.135157
2,2.671196,0.014147
3,2.026101,0.758439
4,2.026101,0.160168
